In [1]:
!pip install svgpath2mpl

In [2]:
from collections import namedtuple
import numpy as np
import torch
from svgpath2mpl import parse_path
import matplotlib.pyplot as plt
from scipy.ndimage import rotate, shift
from matplotlib.animation import FuncAnimation
from probabilistic_fire_env import ProbabilisticFireEnv
from drone_env import DronesEnv
from collections import  deque
from networks.ppo_net import PPONet
from torch.distributions import Categorical
import torch.nn.functional as F


In [3]:
DT          = 0.5  # Time between wildfire updates            
DTI         = 0.1  # Time between aircraft decisions
n_actions = 2
height = width = 100
channels = 2
EPISODES_PER_BATCH = 1
TRAIN_FREQ  = 10
SAVE_FREQ = 10
GAMMA = 0.95
CLIP  = 0.2
BATCH_SIZE = 64
LAMDA = 0.95
UPDATES = 10000
STEP_PER_TRAIN = 512
EPSILON = 0.2
BATCH_SIZE = 16
EPOCHS = 10
NUM_PROCESSES = 8
BETA = 0.1
TAU = 1.0


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PPONet(device,  channels, height, width, n_actions).to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

cov_var = torch.full(size=(n_actions,), fill_value=0.5, device=device)
cov_mat = torch.diag(cov_var)

TypeError: __init__() missing 1 required positional argument: '_outputs'

In [ ]:
Transition = namedtuple('Transition', ('belief_map', 'state_vector', 'action', 'reward', 'value', 'log_policy', 'done'))

class ReplayMemory:

  def __init__(self, capacity):
    self.capicity = capacity
    self._memory = deque([],maxlen=self.capicity)

  def push(self, *args):
    """Save a transition"""
    self._memory.append(Transition(*args))

  @property
  def transitions(self):
    return self._memory

  def __len__(self):
    return len(self.memory)

In [ ]:
DT          = 0.5  # Time between wildfire updates            
DTI         = 0.1  # Time between aircraft decisions
fireEnv = ProbabilisticFireEnv(height, width)
dronesEnv = DronesEnv(height, width, DT, DTI) 
loss = None
i_episode = 1
N_DRONES = 2
steps = 0
done = False


In [ ]:
seed = fireEnv.reset()
dronesEnv.reset(seed)

memory = [ReplayMemory(3000), ReplayMemory(3000)]
state_vectors = [None]*N_DRONES
maps = [None]*N_DRONES
steps_for_episode = 0 

while True:
    
    for j in range(TRAIN_FREQ//int(2*DT/DTI)):

        observation = fireEnv.step()
        
        for d in range(N_DRONES):

            state_vectors[d] = dronesEnv.drones[d].state
            maps[d] = dronesEnv.drones[d].observation

            state_vectors[d] = torch.tensor(state_vectors[d], device=device, dtype=torch.float)
            maps[d] = torch.tensor(maps[d], device=device, dtype=torch.float)



        for i in range(int(DT/DTI)):
            
            steps += 1

            policy = [None]*N_DRONES
            values = [None]*N_DRONES
            actions = [None]*N_DRONES
            old_m = [None]*N_DRONES
            old_log_policy = [None]*N_DRONES
            rewards = [None]*N_DRONES

            for d in range(N_DRONES):

                policy[d], values[d] = model(maps[d], state_vectors[d])
                
            
                values[d] = values[d].squeeze()
        
            
                old_m[d] = Categorical(policy[d])
    

                actions[d] = old_m.sample()

            
                old_log_policy[d] = old_m[d].log_prob(actions[d])



            rewards = dronesEnv.step(actions, observation)
            done = not fireEnv.fire_in_range(6)
            for d in range(N_DRONES):

                state_vectors[d] = dronesEnv.drones[d].state
                maps[d] = dronesEnv.drones[d].observation

                state_vectors[d] = torch.tensor(state_vectors[d], device=device, dtype=torch.float)
                maps[d] = torch.tensor(maps[d], device=device, dtype=torch.float)
                memory[d].push(map[d], state_vectors[d], actions[d], rewards[d], values[d], old_log_policy[d], done)

            if done:
                i_episode += 1
                seed = fireEnv.reset()
                dronesEnv.reset(seed)

            if steps % STEP_PER_TRAIN == 0:

                for d in range(N_DRONES):
                    _, next_value, = model(maps[d], state_vectors[d])
                    next_value = next_value.squeeze()
                    batch  = Transition(*zip(*memory[d].transitions))


                    old_log_policies_batch = torch.cat(batch.log_policy).detach()
               

                    actions_batch = torch.cat(batch.action)


                    value_batch = torch.cat(batch.value).detach()


                    belief_map_batch = torch.cat(batch.belief_map)


                    state_vector_batch = torch.cat(batch.state_vector)


                    reward_batch = batch.reward 

    
                    done_batch = batch.done

                
                    gae = 0
                    R = []



                    for value, reward, done in list(zip(value_batch, reward_batch, done_batch))[::-1]:
                        gae = gae * GAMMA * TAU
                        gae = gae + reward + GAMMA * next_value.detach() * (1 - done) - value.detach()
                        next_value = value
                        R.append(gae + value)

                    R = R[::-1]
                    R = torch.cat(R).detach()
                    advantages = R - value_batch

                    for e_i in range(EPOCHS):
                        indice = torch.randperm(NUM_PROCESSES * STEP_PER_TRAIN)
                        for b_i in range(BATCH_SIZE):
                            batch_indices = indice[
                                int(j * (STEP_PER_TRAIN * NUM_PROCESSES / BATCH_SIZE)): int((j + 1) * (
                                        STEP_PER_TRAIN * NUM_PROCESSES / BATCH_SIZE))]
                            new_policy, value = model(belief_map_batch[batch_indices], state_vector_batch[batch_indices])
                            new_m = Categorical(new_policy)
                            new_log_policy = new_m.log_prob(actions_batch[batch_indices])
                            ratio = torch.exp(new_log_policy - old_log_policies_batch[batch_indices])
                            ratio = torch.exp(new_log_policy - old_log_policies_batch[batch_indices])

                            actor_loss = -torch.mean(
                                torch.min(ratio * advantages[batch_indices],
                                    torch.clamp(ratio, 1.0 - EPSILON, 1.0 + EPSILON) *
                                    advantages[batch_indices]
                                )
                            )

                            critic_loss = F.smooth_l1_loss(R[batch_indices], value_batch.squeeze())
                            entropy_loss = torch.mean(new_m.entropy())
                            total_loss = actor_loss + critic_loss - BETA * entropy_loss
                            optimizer.zero_grad()
                            total_loss.backward()
                            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                            optimizer.step()

                print("Episode: {}. Total loss: {}".format(i_episode, total_loss))


AttributeError: 'tuple' object has no attribute 'squeeze'